In [ ]:
pip install -q replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
import time
import replicate
import pandas as pd

In [ ]:
df = pd.read_csv('/content/WELFake_Dataset.csv', nrows=478, on_bad_lines='skip')

In [ ]:
# multiple shots shot
def create_prompt(article_text):
    prompt = f"""
    The following are clues to detect whether news articles are "real" or "fake."
    Articles related to politics ( mainly in USA and middle eastern and asian countries) or national security and defence or terrorism or climate change and those coming from Reuters have higher chance to be fake.
    Articles that cite reputable sources and have image and video evidence in them, are more likely to be real.

    Now Classify the given article as either "real" or "fake." :
    Article: "{article_text}"
    Label:
    """
    return prompt


In [ ]:
client = replicate.Client(api_token="")

In [ ]:
predictions = []

# Iterate over the 500 examples
for index, row in df.iterrows():
    article = row['text']  # assuming the text column is named 'text'
    prompt = create_prompt(article)

    # Call the model for prediction
    result = client.run(
        "ibm-granite/granite-3.0-8b-instruct",
        input={
        "top_k": 10,
        "prompt": prompt,
        "max_tokens": 10,
        "min_tokens": 1,
        "temperature": 0,
        "system_prompt": "You are an expert in fake news detection and your job is to classify articles into fake or real news. Only answer with 'Real' or 'Fake'",
        "repetition_penalty": 1,
        "presence_penalty": 0,
        "frequency_penalty": 0
    },
    )

    # Assuming the result is a string with 'Real' or 'Fake' as output
    predictions.append(result[-1].strip())  # Clean the result (strip spaces)

    # Introduce a delay of 1 second between requests
    time.sleep(0.5)
      # You can adjust the sleep time (in seconds) as needed

print(predictions[:5])  # Print first 5 predictions to check


ReplicateError: ReplicateError Details:
title: Monthly spend limit reached
status: 402
detail: You've hit your monthly spend limit. You can change or remove your limit at https://replicate.com/account/billing#limits.

In [ ]:
predictions

['Fake',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Fake',
 'Fake',
 ', but I need',
 'Real',
 'Real',
 'Real',
 'Real',
 'Real',
 'Fake',
 '',
 'Real',
 '',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Fake',
 'Fake',
 '',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Real',
 'The article',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Fake',
 'Fake',
 'Real',
 'is a',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Fake',
 '',
 '',
 'Real',
 'Real',
 'current state',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Fake',
 'Fake',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Real',
 'Real',
 '',
 '',
 'Real',
 'Fake',
 'Real',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Real',
 'Fake',
 '',
 'Real',
 'Real',
 '',
 'Real',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Real',
 'Real',
 'Real',
 'Fake',
 'Fake',
 'Fake',
 'Real',
 'Real',
 '',
 'Fake',
 'Fake',
 'Fake',
 'confusion, but I need',
 'Real',
 'Fake',
 'Fake',
 'Real',


In [ ]:
len(predictions)

478

In [ ]:
filtered_labels = ['Real' if 'Real' in label else 'Fake' if 'Fake' in label else 'Fake' for label in predictions]

len(filtered_labels)


478

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming 'label' column contains the true labels
true_labels = df['label'].replace({1: 'Real', 0: 'Fake'}).values  # Replace with the actual column name for true labels
#true_labels = true_labels[:500]
# Calculate accuracy
accuracy = accuracy_score(true_labels, filtered_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 23.64%


In [ ]:
full_predictions = ['Real' if 'Real' in label else 'Fake' if 'Fake' in label else None for label in predictions]


In [ ]:
df = pd.DataFrame({'True Labels': true_labels, 'Predictions': full_predictions})

# Remove rows where any column is None (or NaN)
df_cleaned = df.dropna()

In [ ]:
accuracy = accuracy_score(df_cleaned['True Labels'], df_cleaned['Predictions'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 21.14%
